In [5]:
import random
import numpy as np

# Define Personas and their Purchase Habits
personas = {
    "The Confident Boomer":         {"percentage": 35, "purchases": {"Espresso Drink": 0.4, "Pour Over Coffee": 0.5, "Tea": 0.1, "Sparkling Tea Drink": 0.2, "Pastry": 0.4, "Sandwich": 0.3, "Salad": 0.2, "Soup": 0.1}},
    "The Affluent Professional":    {"percentage": 20, "purchases": {"Espresso Drink": 0.5, "Pour Over Coffee": 0.2, "Pastry": 0.5, "Sandwich": 0.3, "Sparkling Tea Drink": 0.2, "Bubble Tea": 0.2, "Salad": 0.2}},
    "The Family":                   {"percentage": 30, "purchases": {"Bubble Tea": 0.7, "Bubble Tea Upgrades": 0.2, "Kids Drink": 0.4, "Pastry": 0.3, "Sandwich": 0.3, "Popcorn Chicken": 0.4, "Garlic Fries": 0.4, "Waffles": 0.4}},
    "The Socializer":               {"percentage": 8,  "purchases": {"Bubble Tea": 0.5, "Bubble Tea Upgrades": 0.2, "Sparkling Tea Drink": 0.4, "Tea": 0.2, "Garlic Fries": 0.5, "Popcorn Chicken": 0.4, "Waffles": 0.2}},
    "The Health-Conscious":         {"percentage": 7,  "purchases": {"Tea": 0.2, "Espresso Drink": 0.2, "Pour Over Coffee": 0.3, "Sparkling Tea Drink": 0.3, "Sandwich": 0.2, "Pastry": 0.2, "Salad": 0.4, "Soup": 0.3}},
}

# New: Define average prices for each menu item
prices = {
    "Espresso Drink": 6.5,
    "Pour Over Coffee": 5.50,
    "Pastry": 5.5,
    "Sandwich": 11.0,
    "Bubble Tea": 6.5,
    "Bubble Tea Upgrades": .75,
    "Tea": 5.0,
    "Sparkling Tea Drink": 5.5,
    "Kids Drink": 4.0,
    "Waffles": 5.0,
    "Popcorn Chicken": 7.50,
    "Garlic Fries": 6.50,
    "Salad": 13.0,
    "Soup": 8.0,
}

costs = {
    "Espresso Drink": 2.0,
    "Pour Over Coffee": 1.5,
    "Pastry": 2.0,
    "Sandwich": 3.0,
    "Bubble Tea": 2.5,
    "Bubble Tea Upgrades": .05,
    "Tea": 0.25,
    "Sparkling Tea Drink": 1.5,
    "Kids Drink": 1.0,
    "Waffles": 1.0,
    "Popcorn Chicken": 1.46,
    "Garlic Fries": 1.00,
    "Salad": 3.0,
    "Soup": 2.0,
}

item_relations = {
    "Espresso Drink": {"Pastry": 0.2},
    "Pour Over Coffee": {"Pastry": 0.2},
    "Pastry": {"Espresso Drink": 0.1, "Pour Over Coffee": 0.1, "Tea": 0.05, "Bubble Tea": 0.1},
    "Sandwich": {"Espresso Drink": 0.05, "Sparkling Tea Drink": 0.1, "Bubble Tea": 0.1},
    "Bubble Tea": {"Waffles": 0.3},
    "Tea": {"Pastry": 0.1},
    "Sparkling Tea Drink": {"Sandwich": 0.05},
    "Kids Drink": {"Pastry": 0.1},
    "Waffles": {"Bubble Tea": 0.3, "Tea": 0.1},
    "Popcorn Chicken": {"Bubble Tea": 0.2,},
    "Garlic Fries": {"Bubble Tea": 0.1,},
    "Salad": {"Sparkling Tea Drink": 0.1},
    "Soup": {"Sandwich": 0.1, "Sparkling Tea Drink": 0.1},
}

sample_orders_by_persona = {persona: [] for persona in personas.keys()} 

def adjusted_purchase_count():
    mean = 2.7
    std_dev = 1.5  # Adjust this to shape the distribution appropriately
    purchase_count = int(round(np.random.normal(mean, std_dev)))
    return max(1, min(purchase_count, 8))  # Ensure it's within 1 to 8

def adjust_probability(item, selected_items):
    adjustment = 0
    for related_item, effect in item_relations.get(item, {}).items():
        if related_item in selected_items:
            adjustment += effect
    return adjustment

def calculate_purchases_and_revenue(estimated_total_customers):
    total_purchases = {item: 0 for item in prices.keys()}
    total_revenue = {item: 0 for item in prices.keys()}
    total_costs = {item: 0 for item in costs.keys()}  
    total_profit = {item: 0 for item in prices.keys()}  
    
    for persona, data in personas.items():
        num_customers_in_persona = (data['percentage'] / 100) * estimated_total_customers
        
        for _ in range(int(num_customers_in_persona)):
            selected_items = []
            num_items_to_purchase = adjusted_purchase_count()  # Determine number of items to purchase
            
            item_probabilities = list(data['purchases'].items())
            for _ in range(num_items_to_purchase):
                items, probabilities = zip(*item_probabilities)
                adjusted_probabilities = [min(1, p + adjust_probability(item, selected_items)) for item, p in item_probabilities]
                total_prob = sum(adjusted_probabilities)
                normalized_probabilities = [p / total_prob for p in adjusted_probabilities]
                
                item = np.random.choice(items, p=normalized_probabilities)
                total_purchases[item] += 1
                total_revenue[item] += prices[item]
                total_costs[item] += costs[item]
                selected_items.append(item)
    
    for item in total_profit.keys():
        total_profit[item] = total_revenue[item] - total_costs[item]
    
    total_revenue_sum = sum(total_revenue.values())
    total_costs_sum = sum(total_costs.values())
    total_profit_sum = sum(total_profit.values())
    
    return total_purchases, total_revenue, total_costs, total_profit, total_revenue_sum, total_costs_sum, total_profit_sum, sample_orders_by_persona

def calculate_purchases_and_revenue_monte_carlo(estimated_total_customers, num_simulations):
    # Initialize accumulators for total purchases, revenue, costs, and profit for each item
    item_purchases_sums = {item: 0 for item in prices.keys()}
    simulations_revenue = []
    simulations_costs = []
    simulations_profit = []
    
    for i in range(num_simulations):
        # Run the simulation and unpack all the additional return values properly
        total_purchases, total_revenue, total_costs, total_profit, total_revenue_sum, total_costs_sum, total_profit_sum, _ = calculate_purchases_and_revenue(estimated_total_customers)
        
        # Accumulate total purchases, revenue, costs, and profit for each item
        for item in total_purchases.keys():
            item_purchases_sums[item] += total_purchases[item]
        simulations_revenue.append(total_revenue_sum)
        simulations_costs.append(total_costs_sum)
        simulations_profit.append(total_profit_sum)
        
        # Optionally log completion of each simulation
        if (i+1) % 1000 == 0 or i == num_simulations - 1:
            print(f"Simulation {i+1}/{num_simulations} completed.")
    
    # Calculate the average total purchases, revenue, costs, and profit across all simulations
    avg_item_purchases = {item: total / num_simulations for item, total in item_purchases_sums.items()}
    avg_total_revenue = sum(simulations_revenue) / num_simulations
    avg_total_costs = sum(simulations_costs) / num_simulations
    avg_total_profit = sum(simulations_profit) / num_simulations
    
    return avg_item_purchases, avg_total_revenue, avg_total_costs, avg_total_profit

estimated_total_customers = 2500
num_simulations = 1000
# Adjusted call to the Monte Carlo simulation function
avg_item_purchases, avg_total_revenue, avg_total_costs, avg_total_profit = calculate_purchases_and_revenue_monte_carlo(estimated_total_customers, num_simulations)

avg_total_revenue_per_year = avg_total_revenue * 12

# Displaying the results
print("Average Purchases for Each Menu Item:")
for item, avg_purchases in avg_item_purchases.items():
    print(f"{item}: {avg_purchases:.2f}")
print(f"\nAverage Total Revenue per Month: ${avg_total_revenue:.2f}")
print(f"Average Total Costs per Month: ${avg_total_costs:.2f}")
print(f"Average Gross Margin (Revenue - COGS) per Month: ${avg_total_profit:.2f}")

print(f"\nAverage Total Revenue per Year: ${avg_total_revenue_per_year:.2f}")

print('--------- Sample Orders --------- ')

# Iterate through each persona to display their sample orders
for persona, orders in sample_orders_by_persona.items():
    print(f"\nSample Orders for {persona}:")
    for order_index, order in enumerate(orders, start=1):
        print(f"  Order {order_index}: {order}")


Simulation 1000/1000 completed.
Average Purchases for Each Menu Item:
Espresso Drink: 841.89
Pour Over Coffee: 774.36
Pastry: 1052.64
Sandwich: 785.03
Bubble Tea: 647.14
Bubble Tea Upgrades: 170.00
Tea: 210.20
Sparkling Tea Drink: 499.44
Kids Drink: 259.16
Waffles: 348.78
Popcorn Chicken: 374.06
Garlic Fries: 378.53
Salad: 437.95
Soup: 196.62

Average Total Revenue per Month: $47600.72
Average Total Costs per Month: $12973.44
Average Gross Margin (Revenue - COGS) per Month: $34627.28

Average Total Revenue per Year: $571208.61
--------- Sample Orders --------- 

Sample Orders for The Confident Boomer:

Sample Orders for The Affluent Professional:

Sample Orders for The Family:

Sample Orders for The Socializer:

Sample Orders for The Health-Conscious:
